# 목표

- 뉴스를 수집(확보)해서 데이터 분석을 하고자 한다
- 해당 뉴스를 5분단위로 계속 수집하여, 언론사별로 뉴스가 나오는 빈도, 기자별로 뉴스를 수정하는 빈도, 뉴스의 유통량등등 분석하여, 뉴스가 언론 형성에 어느 정도 영향을 미치는지 분석하고 싶다
  - 뉴스를 지속적으로 수집하여야 한다
  - 데이터가 **뉴스**이다
- 포털 사이트에서 제공하는 open api를 이용하여(실제로는 다양한 루트를 연구해 봤을것이다) 수집하겟다

- 추가 인사이트
  - 통신 규약 => 프로토콜 => 통신을 수행하는 상호간의 약속 (이렇게 보낼테니, 받아서 처리하세요:신뢰)
    - http, https, ftp, ...
  - TCP/IP 기반(신뢰기반, 손실없음), UDP 기반(100개 보냈는데 90개만 왔다 OK, 스트리밍)
  - **TCP/IP 기반위에 http가 존재한다 -> 정보 손실이 없다(통신중에)**

- 필요한 재료
  - naver 기준
  - open api 신청
  - url등 사용법을 알아야 사용 가능
  - https://developers.naver.com/main/

# 코드

## 모듈 가져오기

In [2]:
# 운영체계 관련 모듈
import os
# 시스템 모듈
import sys
# 인코딩및 통신처리를 위해서 필요한 모듈
import urllib.request

## 인증키 정의

In [3]:
# 네이버 openapi를 사용하기 위한 인증키
# 파이썬은 상수가 없지만, 변수명을 대문자로 표현해서
# 개발자에게, 참고하는 제 3자에게 해당 내용은 수정하지 말라고 표현한것이다
# 관습적인 표현
# 상수, 대문자 => 환경변수, 시스템변수, 실험환경을 통제하는 임계값등등

CLIENT_ID     = 'quAs4Y6uHPoWLrrwpi0W'
CLIENT_SECRET = 'vqVSw4zlV3'

## 검색어 정의

In [4]:
# 뉴스 검색어
# 검색어 -> 한글도 가능, 영어, 숫자, 특수문자등등...
# abcABD가나다123!@#
# 한글을 서버로 전송하면 기본적으로 한글이 깨진다 -> 인코딩 처리가 필요하다
searchKeyword = '한미정상회담'
searchKeyword

'한미정상회담'

## 검색어 인코딩

In [5]:
# urllib.parse.quote() => 인코딩 처리하는 함수
encText = urllib.parse.quote( searchKeyword )
encText
# 숫자 알파벳은 그대로 나옴
# %ED%95%9C%EB%AF%B8%EC%A0%95%EC%83%81%ED%9A%8C%EB%8B%B412aB%21%40

'%ED%95%9C%EB%AF%B8%EC%A0%95%EC%83%81%ED%9A%8C%EB%8B%B4'

- 일반적인 S/W에서 데이터 포멧
  - 서버와 클라이언트간에 뭔가 통신을 한다 
  - 이때, 보편적으로 가장 많이 사용하는 데이터 형식이 텍스트 포멧이다
    - 규칙을 부여
      - xml(1999), json(2008~)등등..
      - json이 더 저용량으로 데이터를 표현할수 있다
      - **json => 파이썬에서 딕셔너리, 리스트**

## 접속 URL 정의, 세팅

In [6]:
# 뉴스를 네이버에 요청해서 결과를 가져올수 있는 URL이 필요
url = 'https://openapi.naver.com/v1/search/news.json'
url

'https://openapi.naver.com/v1/search/news.json'

## 요청 URL 최종 정의

In [7]:
# 데이터(파라미터)를 서버에 전달하는 방법? Method
# Method => Get, Post, Put, Delete,...
# URL?키=값&키2=값2(데이터,검색어,파라미터)
# 데이터를 여러개 보내면 &를 구분자로 사용한다
# URL?query=값(데이터,검색어,파라미터)
# 문자열 포멧팅 %, format(), f'' 
# 네이버야 특정 키워드로 뉴스를 검색요청하니 검색하고 20개 결과보내줘
url = f'{url}?query={encText}&display=20'
url
# https://openapi.naver.com/v1/search/news.json?query=%3E....

'https://openapi.naver.com/v1/search/news.json?query=%ED%95%9C%EB%AF%B8%EC%A0%95%EC%83%81%ED%9A%8C%EB%8B%B4&display=20'

## 요청객체 생성

In [17]:
# 요청 객체 생성한다, 클레스는 첫글자가 대문자
request = urllib.request.Request(url)
# 헤더에 인증키를 추가한다(최대한 숨겨서 서버로 전송)
request.add_header("X-Naver-Client-Id",CLIENT_ID)
request.add_header("X-Naver-Client-Secret",CLIENT_SECRET)

In [18]:
# 요청 -> 인터넷 -> 서버 -> 내부과정 -> 응답
response = urllib.request.urlopen(request)

## 요청

## 응답처리 (파싱)

In [19]:
# json 형식으로 전달된 데이터(수치, 문자열, 불린)를 
# 처리하기 위한 모듈
import json

In [11]:
# 응답 결과 확인
rescode = response.getcode()
# 응답 코드 => http를 이용하여 요청을 했을때 서버가 응답하는 코드
# 404 => 그런페이지 없다
# 500 => 서버 오류
# 403 or 401 => 권한없다
# 200 => 응답 성공
if(rescode==200):
    # 응답 결과를 딕셔너리 객체로 만든다
    # json => 딕셔너리로 만들어는 모듈이 있을것이다
    # 응답 결과가 잘 도착했으므로 데이터를 추출하기 위해서 
    # json.load() 라는 함수를 이용하여 특정 객체(딕셔너리)로 추출한다
    tmp = json.load( response )
    # 헤드라인만 뽑아본다 -> title
    #response_body = response.read()
    #print(response_body.decode('utf-8'))
else:
    print("Error Code:" + rescode)

In [25]:
# 딕셔너리의 인덱싱
# 변수[ 키 ]
#type(tmp['items']), len(tmp['items'])
for item in tmp['items']:
  #print( type(item) )
  # <b>, </b>, &quot;
  print( item["title"].replace('&quot;','').replace('<b>','').replace('</b>','') )

# 실습
# 제목만 뽑아서 titles라는 리스트에 담고, 출력하시오
titles

두산중공업 주가 5%↑...文·바이든 한미 원전 시장서 협력 강화 발언에 강...
여영국 “한미정상회담으로 대북정책 우려 해소···남북관계 급물쌀 탈 수 ...
김정은, 1만 자 서한으로 '경제 발전' 주문…대외 메시지는 없어
美 전직 대사들 한미 정상회담 민주 가치 강조, 동맹 정체성과 깊이 보여 줘...
탁현민 文 '참전용사 앞 무릎'은 연출 아닌 美측 즉석 제안
문대통령, 여야정 상설 협의체 실현, 국민들 정치 신뢰할 것
켄코아에어로스페이스 주가 3%↑…왜 갑자기 급등?
박지원 국정원장, 뉴욕 도착…방미 일정 시작
여영국 정의당 대표 박지원 방미…남북 급물살 탈 수도
[이슈 핫] ‘대북통’ 박지원, 전격 미국행 이유는?
한국, 미국 달 탐사 프로젝트 '아르테미스 약정'...10번째 가입국
[특징주] 두산중공업 5% 상승 '원전 협력·폐배터리 기술 개발'
한미 해외 원전 수주 협력 합의, 황금알 낳는 거위일까?
文 대통령·여야 대표, 1년 3개월 만의 회동
[로로쌤의 원포인트 투자] 모더나와 계약에도 하락한 삼바, 실망하긴 이르다
여영국 “국정원장 방미와 한미훈련 연기, 남북 급물살 可”
한미정상회담 문 대통령 '여성 기자'발언 팩트체크
[주식 담는 뱅커] 유철종 농협은행 혼합자산운용 과장
[특징주]두산중공업, 8거래일 연속 `상승`
모두 놀란 한미 ‘노마스크’ 정상회담…탁현민 “바이든이 결심했다”


In [30]:
a = list()
b = []
type(a),type(b)
a.append('hi')
a.append('hi2')
a

['hi', 'hi2']

In [31]:
# 리스트 형태의 그릇을 먼저 준비한다
titles = list()
for item in tmp['items']:
  # 제목을 추출해서 그릇에 하나씩 담는다 => 리스트에 맴버추가  
  titles.append( item["title"].replace('&quot;','').replace('<b>','').replace('</b>','') )

# 실습
# 제목만 뽑아서 titles라는 리스트에 담고, 출력하시오
titles

['두산중공업 주가 5%↑...文·바이든 한미 원전 시장서 협력 강화 발언에 강...',
 '여영국 “한미정상회담으로 대북정책 우려 해소···남북관계 급물쌀 탈 수 ...',
 "김정은, 1만 자 서한으로 '경제 발전' 주문…대외 메시지는 없어",
 '美 전직 대사들 한미 정상회담 민주 가치 강조, 동맹 정체성과 깊이 보여 줘...',
 "탁현민 文 '참전용사 앞 무릎'은 연출 아닌 美측 즉석 제안",
 '문대통령, 여야정 상설 협의체 실현, 국민들 정치 신뢰할 것',
 '켄코아에어로스페이스 주가 3%↑…왜 갑자기 급등?',
 '박지원 국정원장, 뉴욕 도착…방미 일정 시작',
 '여영국 정의당 대표 박지원 방미…남북 급물살 탈 수도',
 '[이슈 핫] ‘대북통’ 박지원, 전격 미국행 이유는?',
 "한국, 미국 달 탐사 프로젝트 '아르테미스 약정'...10번째 가입국",
 "[특징주] 두산중공업 5% 상승 '원전 협력·폐배터리 기술 개발'",
 '한미 해외 원전 수주 협력 합의, 황금알 낳는 거위일까?',
 '文 대통령·여야 대표, 1년 3개월 만의 회동',
 '[로로쌤의 원포인트 투자] 모더나와 계약에도 하락한 삼바, 실망하긴 이르다',
 '여영국 “국정원장 방미와 한미훈련 연기, 남북 급물살 可”',
 "한미정상회담 문 대통령 '여성 기자'발언 팩트체크",
 '[주식 담는 뱅커] 유철종 농협은행 혼합자산운용 과장',
 '[특징주]두산중공업, 8거래일 연속 `상승`',
 '모두 놀란 한미 ‘노마스크’ 정상회담…탁현민 “바이든이 결심했다”']

In [33]:
# 리스트 내포 => 심플하게 구성 => 최종 결과에 맞춰서 작성
# titles = list()
# for item in tmp['items']:
#   titles.append( item["title"].replace('&quot;','').replace('<b>','').replace('</b>','') )

titles = [ 
 item["title"].replace('&quot;','').replace('<b>','').replace('</b>','')
 for item in tmp['items']
]
titles

['두산중공업 주가 5%↑...文·바이든 한미 원전 시장서 협력 강화 발언에 강...',
 '여영국 “한미정상회담으로 대북정책 우려 해소···남북관계 급물쌀 탈 수 ...',
 "김정은, 1만 자 서한으로 '경제 발전' 주문…대외 메시지는 없어",
 '美 전직 대사들 한미 정상회담 민주 가치 강조, 동맹 정체성과 깊이 보여 줘...',
 "탁현민 文 '참전용사 앞 무릎'은 연출 아닌 美측 즉석 제안",
 '문대통령, 여야정 상설 협의체 실현, 국민들 정치 신뢰할 것',
 '켄코아에어로스페이스 주가 3%↑…왜 갑자기 급등?',
 '박지원 국정원장, 뉴욕 도착…방미 일정 시작',
 '여영국 정의당 대표 박지원 방미…남북 급물살 탈 수도',
 '[이슈 핫] ‘대북통’ 박지원, 전격 미국행 이유는?',
 "한국, 미국 달 탐사 프로젝트 '아르테미스 약정'...10번째 가입국",
 "[특징주] 두산중공업 5% 상승 '원전 협력·폐배터리 기술 개발'",
 '한미 해외 원전 수주 협력 합의, 황금알 낳는 거위일까?',
 '文 대통령·여야 대표, 1년 3개월 만의 회동',
 '[로로쌤의 원포인트 투자] 모더나와 계약에도 하락한 삼바, 실망하긴 이르다',
 '여영국 “국정원장 방미와 한미훈련 연기, 남북 급물살 可”',
 "한미정상회담 문 대통령 '여성 기자'발언 팩트체크",
 '[주식 담는 뱅커] 유철종 농협은행 혼합자산운용 과장',
 '[특징주]두산중공업, 8거래일 연속 `상승`',
 '모두 놀란 한미 ‘노마스크’ 정상회담…탁현민 “바이든이 결심했다”']

In [22]:
type(tmp)
tmp['items']

[{'description': '최근 진행됐던 <b>한미 정상회담</b>에서 문재인 대통령과 조 바이든 미국 대통령은 &quot;원전 사업 공동 참여를 포함해 해외 원전 시장에서 협력을 강화한다&quot;고 말한 후 원전 관련주들이 들썩이기 시작했다. 이에 지난... ',
  'link': 'http://www.ajunews.com/view/20210527100823529',
  'originallink': 'http://www.ajunews.com/view/20210527100823529',
  'pubDate': 'Thu, 27 May 2021 10:14:00 +0900',
  'title': '두산중공업 주가 5%↑...文·바이든 &quot;<b>한미</b> 원전 시장서 협력 강화&quot; 발언에 강...'},
 {'description': '이번 <b>한미정상회담</b>으로 그런 우려들이 해소됐다”며 이같이 말했다. 그는 “(문 대통령이) 4·27 판문점 선언, 싱가포르 북미<b>회담</b>의 정신을 원칙적으로 존중한다는 것을 상당한 성과로 강조했는데 저희들이 봐도 출발은... ',
  'link': 'https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=100&oid=011&aid=0003915373',
  'originallink': 'https://www.sedaily.com/NewsView/22MIU7NZJV',
  'pubDate': 'Thu, 27 May 2021 10:13:00 +0900',
  'title': '여영국 “<b>한미정상회담</b>으로 대북정책 우려 해소···남북관계 급물쌀 탈 수 ...'},
 {'description': "지난주 열린 <b>한미 정상회담</b> 등 대외 동향에 대한 언급과 메시지는 나오지 않았다. 김 총비서는 '직맹은 사회주의건설의 새로운 고조기를 앞장에서 열어나가는 전위부대가 되자'라는 제목의 서한에서 &quot;시대와 혁명 앞에... ",
  'lin

In [13]:
# 실습 : tmp에서 title을 있는대로 다 추출하여 출력하시오
type(tmp),  tmp

(dict,
 {'display': 20,
  'items': [{'description': '최근 진행됐던 <b>한미 정상회담</b>에서 문재인 대통령과 조 바이든 미국 대통령은 &quot;원전 사업 공동 참여를 포함해 해외 원전 시장에서 협력을 강화한다&quot;고 말한 후 원전 관련주들이 들썩이기 시작했다. 이에 지난... ',
    'link': 'http://www.ajunews.com/view/20210527100823529',
    'originallink': 'http://www.ajunews.com/view/20210527100823529',
    'pubDate': 'Thu, 27 May 2021 10:14:00 +0900',
    'title': '두산중공업 주가 5%↑...文·바이든 &quot;<b>한미</b> 원전 시장서 협력 강화&quot; 발언에 강...'},
   {'description': '이번 <b>한미정상회담</b>으로 그런 우려들이 해소됐다”며 이같이 말했다. 그는 “(문 대통령이) 4·27 판문점 선언, 싱가포르 북미<b>회담</b>의 정신을 원칙적으로 존중한다는 것을 상당한 성과로 강조했는데 저희들이 봐도 출발은... ',
    'link': 'https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=100&oid=011&aid=0003915373',
    'originallink': 'https://www.sedaily.com/NewsView/22MIU7NZJV',
    'pubDate': 'Thu, 27 May 2021 10:13:00 +0900',
    'title': '여영국 “<b>한미정상회담</b>으로 대북정책 우려 해소···남북관계 급물쌀 탈 수 ...'},
   {'description': "지난주 열린 <b>한미 정상회담</b> 등 대외 동향에 대한 언급과 메시지는 나오지 않았다. 김 총비서는 '직맹은 사회주의건설의 새로운 고조기를 앞장에서 

In [14]:
{
"lastBuildDate": "Wed, 26 May 2021 15:33:01 +0900",
"total": 103930,
"start": 1,
"display": 10,
"items": [
{
"title": "여야 5당 대표 만난 文 &quot;평화 시계 다시 가동… 北 호응 기대&quot;",
"originallink": "http://www.newscj.com/news/articleView.html?idxno=863787",
"link": "http://www.newscj.com/news/articleView.html?idxno=863787",
"description": "문 대통령은 이날 청와대에서 열린 여야 5당 대표 초청 간담회에서 &quot;이번 <b>정상회담</b>은 <b>한미</b>동맹이 끊임없이 발전하고 있다는 사실을 재확인할 수 있었던 뜻 깊은 기회였다&quot;며 이같이 밝혔다. 문 대통령은 &quot;공동성명에... ",
"pubDate": "Wed, 26 May 2021 15:28:00 +0900"

},
{
"title": "文, 정당 대표 초청 대화서 <b>한미정상회담</b> 성과 설명",
"originallink": "http://www.newsprime.co.kr/news/article.html?no=545070",
"link": "http://www.newsprime.co.kr/news/article.html?no=545070",
"description": "이번 정당 대표 초청 대화는 <b>한미 정상회담</b>의 결과를 설명하고 초당적 협력을 부탁하기 위해 마련됐다. 문 대통령은 &quot;<b>한미 정상회담</b>은 <b>한미</b>동맹이 끊임없이 발전하고 있다는 사실을 재확인할 수 있었던 뜻깊은 기회였다&quot;며... ",
"pubDate": "Wed, 26 May 2021 15:28:00 +0900"

},
{
"title": "싱하이밍 &quot;북미 서로 양보해야…대북제재는 완화해야&quot;",
"originallink": "https://www.news1.kr/articles/?4318741",
"link": "https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=100&oid=421&aid=0005374245",
"description": "미국이 <b>한미정상회담</b> 이후 공은 북한에게 넘어갔다고 평가하고 있는데 그는 &quot;중국은 대화는 대화를 안하는 것보단 낫다고 생각한다&quot;면서 &quot;양측이 문제를 해결하도록 점진적으로 가는게 좋다&quot;고 반복했다. 한편 이날... ",
"pubDate": "Wed, 26 May 2021 15:28:00 +0900"

},
{
"title": "與, <b>한미</b> 글로벌 백신 파트너십 적극 지원...송영길 “송도, 백신 생산기지로...",
"originallink": "http://www.viva100.com/main/view.php?key=20210526010005598",
"link": "http://www.viva100.com/main/view.php?key=20210526010005598",
"description": "26일 민주당 지도부는 인천 송도 삼성바이오로직스에서 현장 최고위원회를 갖고 <b>한미정상회담</b>에서 맺어진 <b>한미</b>간 글로벌 백신 파트너십을 높게 평가한 뒤 당 차원의 아낌없는 지원을 약속했다. 송 대표는 모두 발언을... ",
"pubDate": "Wed, 26 May 2021 15:28:00 +0900"

},
{
"title": "안철수 &quot;<b>한미정상회담</b>, 대통령 노고 컸지만 '기대와 우려' 공존&quot;",
"originallink": "http://news.tf.co.kr/read/ptoday/1863699.htm",
"link": "https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=100&oid=629&aid=0000085359",
"description": "&quot;<b>한미</b>동맹 복원·미사일 지침 종료 의미…백신 협력 아쉬워&quot; 안철수 국민의당 대표가 26일 문재인 대통령을 만난 자리에서 &quot;<b>한미 정상회담</b>에서 노고가 크셨다는 말씀드리고 싶다&quot;라면서도 &quot;이번 <b>정상회담</b>에서 기대와... ",
"pubDate": "Wed, 26 May 2021 15:27:00 +0900"

},
{
"title": "[현장연결] 더불어민주당, 청와대 회동 결과 발표",
"originallink": "http://www.yonhapnewstv.co.kr/MYH20210526011700038/?did=1825m",
"link": "https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=100&oid=422&aid=0000486880",
"description": "이번 <b>한미 정상회담</b>의 준비 상황 그다음에 성과, 향후 과제, 전망 이런 것에 대해서 보고를 하고 식사를 하시면서 김기현 국민의힘 대표부터 시작해서 대표님께서 진지하게. 사실 오늘 회동은 문 대통령 방미 성과 관련된... ",
"pubDate": "Wed, 26 May 2021 15:26:00 +0900"

},
{
"title": "[이시각 주요뉴스(15:00)]1분기 혼인 17.6% 줄고… 이혼은 3.5% 늘었다 外",
"originallink": "http://www.seoulwire.com/news/articleView.html?idxno=443929",
"link": "http://www.seoulwire.com/news/articleView.html?idxno=443929",
"description": "사진=픽사베이 ▲1분기 혼인 17.6% 줄고… 이혼은 3.5% 늘었다/조선일보 ▲백악관 &quot;中 코로나 유출설 규명해야&quot;…트럼프 &quot;내가 말하지 않았나&quot;/중앙일보 ▲文 &quot;<b>한미정상회담</b>서 달라진 한국 위상 실감…국회 초당적 협력... ",
"pubDate": "Wed, 26 May 2021 15:26:00 +0900"

},
{
"title": "[속보] 文대통령 &quot;백신스와프 문제는 <b>한미정상회담</b> 의제 아니었다&quot;",
"originallink": "https://www.news1.kr/articles/?4318752",
"link": "https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=100&oid=421&aid=0005374233",
"description": "",
"pubDate": "Wed, 26 May 2021 15:26:00 +0900"

},
{
"title": "이낙연, '양질의 청년 일자리 확대' 위해 중견기업수 2026년까지 7000개 이상...",
"originallink": "http://www.weeklytoday.com/news/articleView.html?idxno=369155",
"link": "http://www.weeklytoday.com/news/articleView.html?idxno=369155",
"description": "이 전 대표는 26일 정태호 의원과 산업통상자원부가 공동주최한 '중견기업 청년 일자리 진단 및 청년 고용 확대 방안 토론회'에 참석해 &quot;<b>한미정상회담</b>이 성공적으로 마무리되면서 <b>한미</b>관계는 안보동맹에서 밸류체인... ",
"pubDate": "Wed, 26 May 2021 15:26:00 +0900"

},
{
"title": "문대통령 &quot;쿼드 중요성 인지…시진핑 방한 미정&quot;",
"originallink": "http://yna.kr/AKR20210526122000001?did=1195m",
"link": "https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=100&oid=001&aid=0012417843",
"description": "문 대통령은 이날 <b>한미정상회담</b> 성과를 공유하기 위한 청와대 5당 대표 오찬 간담회에서 이같이 말하고 &quot;어떤 국가와도 개방성, 투명성을 토대로 사안별 협력할 것들을 먼저 실행하겠다&quot;고 밝혔다고 국민의당 안혜진... ",
"pubDate": "Wed, 26 May 2021 15:24:00 +0900"

}
]
}

{'display': 10,
 'items': [{'description': '문 대통령은 이날 청와대에서 열린 여야 5당 대표 초청 간담회에서 &quot;이번 <b>정상회담</b>은 <b>한미</b>동맹이 끊임없이 발전하고 있다는 사실을 재확인할 수 있었던 뜻 깊은 기회였다&quot;며 이같이 밝혔다. 문 대통령은 &quot;공동성명에... ',
   'link': 'http://www.newscj.com/news/articleView.html?idxno=863787',
   'originallink': 'http://www.newscj.com/news/articleView.html?idxno=863787',
   'pubDate': 'Wed, 26 May 2021 15:28:00 +0900',
   'title': '여야 5당 대표 만난 文 &quot;평화 시계 다시 가동… 北 호응 기대&quot;'},
  {'description': '이번 정당 대표 초청 대화는 <b>한미 정상회담</b>의 결과를 설명하고 초당적 협력을 부탁하기 위해 마련됐다. 문 대통령은 &quot;<b>한미 정상회담</b>은 <b>한미</b>동맹이 끊임없이 발전하고 있다는 사실을 재확인할 수 있었던 뜻깊은 기회였다&quot;며... ',
   'link': 'http://www.newsprime.co.kr/news/article.html?no=545070',
   'originallink': 'http://www.newsprime.co.kr/news/article.html?no=545070',
   'pubDate': 'Wed, 26 May 2021 15:28:00 +0900',
   'title': '文, 정당 대표 초청 대화서 <b>한미정상회담</b> 성과 설명'},
  {'description': '미국이 <b>한미정상회담</b> 이후 공은 북한에게 넘어갔다고 평가하고 있는데 그는 &quot;중국은 대화는 대화를 안하는 것보단 낫다고 생각한다&quot;면서 &quot;양측이 문